# Lotebook for different types of Stepsize Control

The goal of ths spreadsheed is too test methods for encorpirating the adaption of b into the stepsize control.

The used stepsize control is based on 'hairerODEs2' 

$$\Delta t_{n+1} = \Delta t_{n} \left(\frac{Tol}{|err_n|} \right)^{n_1} \left(\frac{|err_{n-1}|}{|err_n|} \right)^{n_2}$$

(it can be translated ito the $\alpha$ $\beta$ notation using $\alpha = n_1 + n_2$ and $\beta = n_2$)

## General Notes

At first it is important to encorporate a stepsize reduction if the LP-Problem of the last step was infeasible. If not done so the new $\Delta t$ can stay high if a small error was calculated and the computation will stall at this point.

Another important observation for the used problem is that $change$ correlates with $err$ in most cases. There are sometimes large deviations but these can be filters using a upper bound on the acceptable change.

Apparently the relation between $t$, $\Delta t $ anf $change$ is not very smoth. (This is not supprising because the appearance of a negative value in the solution is a binary proccess) 
Because of this it is not easy to encorporate $change$ into the stepsize control.

## Ways to encorporate the adaption approach

### Using the standard error estimator augumented with a step reduction 

For this approach the standard error estimatior of the truncation error

$$ err_T = || u^{n+1}_{b_{orig}} - u^{n+1}_{b_{hat}} || = || K (b_{orig} - b_{hat}) ||$$

is used. The error $err_T$ is fed directly into the Formula for the stepsize control.

Seperatly from this a second stepsize $\Delta t_{n+1,POS}$ is calculated. This is defined by

$$\Delta t_{n+1,POS} = \begin{cases}
    fac \Delta t_n       & \quad \text{if } \text{Last step lead to a infeasible LP-Problem}\\
    \infty  & \quad \text{else } \text{}
  \end{cases}$$
  
This is doen to reduce the stepsize to prevent it getting stuck in a loop.

Finaly the minimum of both stepsizes is used.

### Encorporating the change

In this approach we estimate the error of the computation using

$$err = |u(t^n)-u^n| = |u(t^n) - (u^n_{b_{orig}+\Delta t K(b-b_{orig})})|$$
$$ \leq \underbrace{|u(t^n)-u^n_{b_{orig}}|}_{\approx err_T}+\underbrace{|\Delta t K(b-b_{orig})|}_{= err_{adapt}}$$

By because $err_{adapt} = \Delta t change$ we can add $cahnge$ to the error estimation to get an estimation of the total error.
Question: What about the $\Delta t$ in the regular formula?

In the implementation the weighting factor is used to controll the influence of the change. If this factor is not small, $\Delta t$ oszillates around the $\Delta t$ at which the original soultions starts to contain negative values.

In a second Implementation the used values were bounded based on the previous values to prevent large spikes from destroing the stability. 



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg

from numba import jit, float64, stencil

fe =rk.loadRKM('FE').__num__()
rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()



#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils 

In [ ]:
def plot_stepsizes(status,slot = None):

    #At first plot the stepsizes according to the evalautions
    dt = np.array(status['dt_calc'])
    tol_met = np.array(status['sc'])
    neg = np.array(status['b'])
    n= np.arange(len(dt))
    plt.plot(n,dt,'-k')
    plt.plot(n[tol_met=='m'],dt[tol_met=='m'],'bx',label ='steps')
    plt.plot(n[tol_met!='m'],dt[tol_met!='m'],'rx',label ='tol not met')
    plt.plot(n[neg=='r'],dt[neg=='r'],'ro',label = 'infeasible')
    plt.xlim(slot)
    plt.grid()
    plt.legend()

    #plt.ylim([0,0.1])

    #plot error and change in commparison
    plt.figure()
    change = np.array(status['change'])
    error = np.array(status['error'])
    old_min = np.array(status['old_min'])
    plt.plot(change,'x',label = 'Change')
    plt.plot(error,'x',label = 'Error')
    plt.plot(old_min,'x' ,label = 'Min')
    plt.xlim(slot)
    #plt.ylim([-0,1])
    plt.grid()
    plt.legend()

In [ ]:
#Test for stepsize
def dt_logic_PI(stepsize_control,dt_old,error,change,success,tol_met):
    print('error:',error)
    beta1 = 0.9/5
    beta2 = 0.3/5
    facmax = 1.2
    facmin = 0.1
    fac = 0.9
    Tol = stepsize_control.a_tol
    dt = dt_old *(Tol/error)**beta1*(dt_logic_PI.error_old[-1]/error)**beta2
    
    #Update storage vaiables
    dt_logic_PI.error_old[:-1] = dt_logic_PI.error_old[1:] 
    dt_logic_PI.error_old[-1] = error
    
    if True or success: 
        return min(facmax*dt_old,max(facmin*dt_old,dt))
    else:
        return dt_old *0.5
    
dt_logic_PI.error_old = np.zeros(3)
    
#Test for stepsize control with change as seperate controller
"""Has the problem, that it reduces teh stepsize sometimes which afterwards leads to a low error,
that intoduces a larg step afterwards. This makes behavour not very stable
"""
def dt_logic_PI_CHG(stepsize_control,dt_old,error,change,success,tol_met):
    print('error:',error)
    beta1 = (0.7/5)
    beta2 = (0.4/5)
    facmax = 1.2
    facmin = 0.1
    fac = 0.9
    #Control Change
    if not success:
        dt_pos = dt_old*0.5
    elif change == 0: #use this seperatly because it would break formula
        dt_pos = np.infty
    else:
        dt_pos = dt_old * (3/change)
    #Control for TOL
    Tol = stepsize_control.a_tol
    dt_tol = dt_old *(Tol/error)**beta1*(dt_logic_PI_CHG.error_old[-1]/error)**beta2
    dt = min(dt_pos,dt_tol)
    
    #Update storage vaiables
    dt_logic_PI_CHG.error_old[:-1] = dt_logic_PI_CHG.error_old[1:] 
    dt_logic_PI_CHG.error_old[-1] = error

    
    if True or success: 
        return min(facmax*dt_old,max(facmin*dt_old,dt))
    else:
        return dt_old *0.5

dt_logic_PI_CHG.error_old = np.zeros(3)
    
#Test for stepsize control only if LP-Problem infeasible
def dt_logic_PI_POS(stepsize_control,dt_old,error,change,success,tol_met):
    print('error:',error)
    beta1 = (0.7/5)
    beta2 = (0.4/5)
    facmax = 1.2
    facmin = 0.1
    fac = 0.9
    #Control Pos
    if not success:
        dt_pos = dt_old*0.5
    else:
        dt_pos = np.infty
    #Control for TOL
    Tol = stepsize_control.a_tol
    dt_tol = dt_old *(Tol/error)**beta1*(dt_logic_PI_POS.error_old[-1]/error)**beta2
    dt = min(dt_pos,dt_tol)
    
    #Update storage vaiables
    dt_logic_PI_POS.error_old[:-1] = dt_logic_PI_POS.error_old[1:] 
    dt_logic_PI_POS.error_old[-1] = error
    
    if True or success: 
        return min(facmax*dt_old,max(facmin*dt_old,dt))
    else:
        return dt_old *0.5
    
dt_logic_PI_POS.error_old = np.zeros(3)
    
#compute a totla error out of error and change and use this as inmut for a single PI controller
def dt_logic_PI_err_total(stepsize_control,dt_old,error,change,success,tol_met):
    print('error:',error)
    beta1 = (0.7/5)
    beta2 = (0.4/5)
    facmax = 1.2
    facmin = 0.1
    fac = 10
    w_change = 0.5
    #compute total error
    if change == None:
        change = np.infty  #maybe set her some other value
    error_tot = error + w_change*change

    #Control
    Tol = stepsize_control.a_tol
    dt = dt_old *(Tol/error_tot)**beta1*(dt_logic_PI_err_total.error_tot_old[-1]/error_tot)**beta2
    
    #Update storage vaiables
    dt_logic_PI_err_total.error_tot_old[:-1] = dt_logic_PI_err_total.error_tot_old[1:] 
    dt_logic_PI_err_total.error_tot_old[-1] = error_tot

    
    if True or success: 
        return min(facmax*dt_old,max(facmin*dt_old,dt))
    else:
        return dt_old *0.5

dt_logic_PI_err_total.error_tot_old = np.zeros(3)

#dampen the cahnge bevorehand to get rid of the spikes
#compute a totla error out of error and change and use this as inmut for a single PI controller
def dt_logic_PI_err_total_dampend(stepsize_control,dt_old,error,change,success,tol_met):
    print('error:',error)
    beta1 = (0.7/5)
    beta2 = (0.4/5)
    facmax = 1.2
    facmin = 0.1
    fac = 0.9
    w_change = 0.5
    #compute total error
    if change == None:
        change = 200  #maybe set her some other value
    
    prediction_change = 1 + dt_logic_PI_err_total_dampend.change_old@np.ones(5)/5 + min(dt_logic_PI_err_total_dampend.change_old@np.array([-1,-1,0,1,1])*1,0) #some estimate of derivative
    change_dampened = min(prediction_change, change)
    
    error_tot_old = dt_logic_PI_err_total_dampend.error_old + w_change * dt_logic_PI_err_total_dampend.change_old
    error_tot = error + w_change*change_dampened

    #Control
    Tol = stepsize_control.a_tol
    dt = dt_old *(Tol/error_tot)**beta1*(error_tot_old[-1]/error_tot)**beta2
    
    #Update storage vaiables
    dt_logic_PI_err_total_dampend.error_old[:-1] = dt_logic_PI_err_total_dampend.error_old[1:] 
    dt_logic_PI_err_total_dampend.error_old[-1] = error
    dt_logic_PI_err_total_dampend.change_old[:-1] = dt_logic_PI_err_total_dampend.change_old[1:] 
    dt_logic_PI_err_total_dampend.change_old[-1] = change_dampened
    
    return min(facmax*dt_old,max(facmin*dt_old,dt))

dt_logic_PI_err_total_dampend.error_old = np.zeros(5)
dt_logic_PI_err_total_dampend.change_old = np.zeros(5)

#dampen the cahnge bevorehand to get rid of the spikes
#compute a totla error out of error and change and use this as inmut for a single PI controller
def dt_logic_PI_err_total_dampend_POS(stepsize_control,dt_old,error,change,success,tol_met):
    print('error:',error)
    beta1 = (0.7/5)
    beta2 = (0.4/5)
    facmax = 1.2
    facmin = 0.1
    fac = 0.9
    w_change = 0.5
    #compute total error
    if change == None:
        change = 200  #maybe set her some other value
    
    prediction_change = 1 + dt_logic_PI_err_total_dampend.change_old@np.ones(5)/5 + min(dt_logic_PI_err_total_dampend.change_old@np.array([-1,-1,0,1,1])*1,0) #some estimate of derivative
    #+1 is to enable a precition even if all recent changes are equal to 0
    print('prediction_change:',prediction_change)
    print('real.      change:',change)
    change_dampened = min(prediction_change, change)

    
    error_tot_old = dt_logic_PI_err_total_dampend_POS.error_old + w_change * dt_logic_PI_err_total_dampend_POS.change_old
    error_tot = error + w_change*change_dampened

    #Control Pos
    if not success:
        dt_pos = dt_old*0.5
    else:
        dt_pos = np.infty
    
    #Control err_tot
    Tol = stepsize_control.a_tol
    dt_tol = dt_old *(Tol/error_tot)**beta1*(error_tot_old[-1]/error_tot)**beta2
    
    
    dt = min(dt_pos,dt_tol)
    
    #Update storage vaiables
    dt_logic_PI_err_total_dampend_POS.error_old[:-1] = dt_logic_PI_err_total_dampend_POS.error_old[1:] 
    dt_logic_PI_err_total_dampend_POS.error_old[-1] = error
    dt_logic_PI_err_total_dampend_POS.change_old[:-1] = dt_logic_PI_err_total_dampend_POS.change_old[1:] 
    dt_logic_PI_err_total_dampend_POS.change_old[-1] = change_dampened
    
    return min(facmax*dt_old,max(facmin*dt_old,dt))

dt_logic_PI_err_total_dampend_POS.error_old = np.zeros(5)
dt_logic_PI_err_total_dampend_POS.change_old = np.zeros(5)

control = StepsizeControl(dt_min = 0,dt_max = np.infty,a_tol = 0.1,r_tol=0.001,f = dt_logic_PI_err_total_dampend
                          ,tol_reqect = 2*0.1)


#def dt_logic_I(stepsize_control,dt_old,error,change,success,tol_met):
#    print('error:',error)
#    facmax = 1.2
#    facmin = 0.1
#    fac = 0.9
#    if success and tol_met: 
#        return dt_old *min(facmax,max(facmin,fac*(1/(error)**(1/4))))
#    else:
#        return dt_old *0.1

In [ ]:
# Brusselator test problem
def f_brusselator(t,u):
    k1 =1
    k2 =1
    k3 =1
    k4 =1
    k5 =1
    k6 =1
    du = np.zeros(6)
    du[0] = -k1*u[0]
    du[1] = -k2*u[1]*u[4]
    du[2] = k3*u[1]*u[4]
    du[3] = k4*u[4]
    du[4] = k1*u[0] - k2*u[1]*u[4] + k3*u[4]**2*u[5] - k4*u[4]
    du[5] = k2*u[1]*u[4] - k3*u[4]**2*u[5]
    return du


u0 = np.array([10.,10.,0.,0.,0.1,0.1])

In [ ]:
dt = 0.03


solver = Solver(rkm = ck5,
               dt = dt,
               t_final = 50,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = cp.MOSEK,
               fail_on_requect=False,
               LP_opts = {'verbose_LP':False})

problem_brusselator = Problem(f=f_brusselator,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

control = StepsizeControl(dt_min = 0,dt_max = np.infty,a_tol = 0.5,r_tol=0.001,f = dt_logic_PI_err_total_dampend
                          ,tol_reqect = 2*0.1)


#t,u,b,KK = RK_variable_b(ssp104,dt,f_prod,u0,t_final=5,b_fixed=False,dumpK=True)
status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_brusselator,stepsize_control=control,
                               verbose=True,dumpK=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
plt.plot(t,u[0,:],'-x',label='1')
plt.plot(t,u[1,:],'-x',label='2')
plt.plot(t,u[2,:],'-x',label='3')
plt.plot(t,u[3,:],'-x',label='4')
plt.plot(t,u[4,:],'-x',label='5')
plt.plot(t,u[5,:],'-x',label='6')
plt.grid()
plt.legend()

In [ ]:
plot_stepsizes(status)

All Approaches work sufficently well if set up with the reigth parameters and a reaction on infeasible LP-Problems is encorportated. 

## Productio Part 



In [ ]:
@jit(float64[:](float64,float64[:]),nopython=True)
def f_prod(t,u):
        du = np.zeros(4)
        du[0] = 0.01*u[1]+0.01*u[2]+0.003*u[3]-(u[0]*u[1])/(0.01+u[0])
        du[1] = (u[0]*u[1])/(0.01+u[0])-0.01*u[1]-0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.05*u[1]
        du[2] = 0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.01*u[2]-0.02*u[2]
        du[3] = 0.05*u[1] + 0.02*u[2]-0.003*u[3]
        return du

@jit(float64[:](float64,float64[:]),nopython=True)
def f_prod_pos(t,u):
        du = np.zeros(4)
        u = np.where(u>-0.005,u,-0.005)

        du[0] = 0.01*u[1]+0.01*u[2]+0.003*u[3]-(u[0]*u[1])/(0.01+u[0])
        du[1] = (u[0]*u[1])/(0.01+u[0])-0.01*u[1]-0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.05*u[1]
        du[2] = 0.5*(1-np.exp(-1.21*u[1]**2))*u[2]-0.01*u[2]-0.02*u[2]
        du[3] = 0.05*u[1] + 0.02*u[2]-0.003*u[3]
        return du



In [ ]:
#u0 = np.array([1.,1.,1.,1.])

u0 = np.array([8.,2.,1.,4.])
dt = 0.03


solver = Solver(rkm = ck5,
               dt = dt,
               t_final = 10,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = 0.5,
               p = [4,3,2],
               theta = [1],
               solver = cp.MOSEK,
               fail_on_requect=False,
               LP_opts = {'verbose_LP':False})

problem_reaction = Problem(f=f_prod,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

control = StepsizeControl(dt_min = 0,dt_max = np.infty,a_tol = 0.1,r_tol=0.001,f = dt_logic_PI_err_total
                          ,tol_reqect = 2*0.1)


#t,u,b,KK = RK_variable_b(ssp104,dt,f_prod,u0,t_final=5,b_fixed=False,dumpK=True)
status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_reaction,stepsize_control=control,
                               verbose=True,dumpK=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
plt.plot(t,u[0,:],label='1')
plt.plot(t,u[1,:],label='2')
plt.plot(t,u[2,:],label='3')
plt.plot(t,u[3,:],label='4')
plt.grid()
plt.legend()

In [ ]:
plt.plot(t[1:],b.T[1:,:])
plt.grid()

In [ ]:
plot_stepsizes(status,slot=[300,350])

# PDE

In [ ]:
# Now we implement the advection Diffusion equation
# To arcieve compatibility with the existing time integrators the u's are stored in a single vector

#u= [u_1^T,u_2^T,u_3^T,u_4^T]^T




@stencil
def kernel_adv(a,dxi):
    return dxi*(a[0]-a[-1])

@stencil
def kernel_dif(a,dxqi):
    return dxqi*(a[-1]-2*a[0]+a[1])

@jit(float64[:](float64[:],float64),nopython=True)
def adv(u,dxi):
    du = kernel_adv(u,dxi)
    
    #Periodic
    du[0]=dxi*(u[0]-u[-1])
    return du

@jit(float64[:](float64[:],float64),nopython=True)
def dif(u,dxqi):
    du = kernel_dif(u,dxqi)

    #Periodic
    du[0] =dxqi*(u[-1]-2*u[0]+u[1])
    du[-1]=dxqi*(u[-2]-2*u[-1]+u[0])
    
    return du


@jit(float64[:](float64,float64[:]),nopython=True)
def f_ADP(t,u):
    a=1e-2
    d=1e-6
    
    du = np.zeros_like(u)
    
    #split u in different parts
    N = len(u)//4
    u1 = u[0:N]
    u2 = u[N:2*N]
    u3 = u[2*N:3*N]
    u4 = u[3*N:4*N]
    
    #calculate Production
    pu1 = 0.01*u2+0.01*u3+0.003*u4-(u1*u2)/(0.01+u1)
    pu2 = -0.01*u2-0.5*(1-np.exp(-1.21*u2**2))*u3-0.05*u2+(u1*u2)/(0.01+u1)
    pu3 = 0.5*(1-np.exp(-1.21*u2**2))*u3-0.01*u3-0.02*u3
    pu4 = 0.05*u2 + 0.02*u3-0.003*u4
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    
    du[0:N] = -a*adv(u1,1/dx)+d*dif(u1,1/dxq) + pu1 
    du[N:2*N] = -a*adv(u2,1/dx)+d*dif(u2,1/dxq) + pu2
    du[2*N:3*N] = -a*adv(u3,1/dx)+d*dif(u3,1/dxq) + pu3
    du[3*N:4*N] = -a*adv(u4,1/dx)+d*dif(u4,1/dxq) + pu4   
    
    return du


#Added a modification to the function that alters the behavior for values <0.005 to make shure that
#there is no second root of the stageeqation with negative us
@jit(float64[:](float64,float64[:]),nopython=True)
def f_ADP_pos(t,u):
    a=1e-2
    d=1e-6
    
    du = np.zeros_like(u)
    
    #split u in different parts
    N = len(u)//4
    u1 = np.where(u[0:N]>-0.005,u[0:N],-0.005)   #
    u2 = np.where(u[N:2*N]>-0.005,u[N:2*N],-0.005)
    u3 = np.where(u[2*N:3*N]>-0.005,u[2*N:3*N],-0.005)
    u4 = np.where(u[3*N:4*N]>-0.005,u[3*N:4*N],-0.005)
    
    
    
    #calculate Production
    pu1 = 0.01*u2+0.01*u3+0.003*u4-(u1*u2)/(0.01+u1)
    pu2 = -0.01*u2-0.5*(1-np.exp(-1.21*u2**2))*u3-0.05*u2+(u1*u2)/(0.01+u1)
    pu3 = 0.5*(1-np.exp(-1.21*u2**2))*u3-0.01*u3-0.02*u3
    pu4 = 0.05*u2 + 0.02*u3-0.003*u4
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    
    du[0:N] = -a*adv(u1,1/dx)+d*dif(u1,1/dxq) + pu1 
    du[N:2*N] = -a*adv(u2,1/dx)+d*dif(u2,1/dxq) + pu2
    du[2*N:3*N] = -a*adv(u3,1/dx)+d*dif(u3,1/dxq) + pu3
    du[3*N:4*N] = -a*adv(u4,1/dx)+d*dif(u4,1/dxq) + pu4   
    
    return du


def precond_ADP(t,u,dt,a,f): #solves the impicit Advection and diffusion Problem without the Production part
    a=1e-2
    d=1e-6
    
    y0 = np.zeros_like(u)
    
    N = len(u)//4
    u1 = u[0:N]
    u2 = u[N:2*N]
    u3 = u[2*N:3*N]
    u4 = u[3*N:4*N]
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    A_dif = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))
    A_dif[-1,0] = 1/dx**2
    A_dif[0,-1] = 1/dx**2

    A_adv = 1/dx * (np.diag(np.ones(N))-np.diag(np.ones(N-1),-1))
    A_adv[0,-1] = 1/dx
    
    A = -a*A_adv + d*A_dif
    
    y0[0:N] = u1+dt*a*solver_Matrix(t,u1,dt,a,A)
    y0[N:2*N] = u2+dt*a*solver_Matrix(t,u2,dt,a,A)
    y0[2*N:3*N] = u3+dt*a*solver_Matrix(t,u3,dt,a,A)
    y0[3*N:4*N] = u4+dt*a*solver_Matrix(t,u4,dt,a,A)  
    
    return y0
    
def precond_ADP_forward(t,u,dt,a,f):
    a=1e-2
    d=1e-6
    
    y0 = np.zeros_like(u)
    
    N = len(u)//4
    u1 = u[0:N]
    u2 = u[N:2*N]
    u3 = u[2*N:3*N]
    u4 = u[3*N:4*N]
    
    #Advection and Diffusion
    dx=1/N #periodic boundary
    dxq = dx**2
    
    A_dif = 1/dx**2 * (-2*np.diag(np.ones(N))+np.diag(np.ones(N-1),-1)+np.diag(np.ones(N-1),1))
    A_dif[-1,0] = 1/dx**2
    A_dif[0,-1] = 1/dx**2

    A_adv = 1/dx * (np.diag(np.ones(N))-np.diag(np.ones(N-1),-1))
    A_adv[0,-1] = 1/dx
    
    A = -a*A_adv + d*A_dif
    
    y0[0:N] = u1+dt*a*A@u1
    y0[N:2*N] = u2+dt*a*A@u2
    y0[2*N:3*N] = u3+dt*a*A@u3
    y0[3*N:4*N] = u4+dt*a*A@u4  
    
    return y0

In [ ]:
#Additional functions for better handling of System

def initial_ADP(N):
    start = np.array([8.,2.,1.,4.])
    u1s = np.zeros(N)
    u1s[0:N//2]=7
    u1s[N//2:]=8

    u2s = np.zeros(N)
    u2s[N//2:-1]=2

    u3s = np.zeros(N)
    u3s[N//2:]=1

    u4s = np.zeros(N)
    u4s[0:N//2]=1
    u4s[N//2:]=4


    u0=np.ones(N*4)
    u0[0:N] =  u1s
    u0[N:2*N] =  u2s
    u0[2*N:3*N] =  u3s
    u0[3*N:4*N] =  u4s
    
    return u0
    

def plot_image(u,t):
    N = u.shape[0]//4
    cmap=plt.get_cmap('plasma')
    display(u.shape)
    extent=[0,t[-1],1,0]
    plt.subplot(4, 1, 1)
    plt.imshow(u[0:N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 2)
    plt.imshow(u[N:2*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 3)
    plt.imshow(u[2*N:3*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')
    plt.subplot(4, 1, 4)
    plt.imshow(u[3*N:4*N,:],cmap=cmap,extent=extent)
    plt.colorbar()
    plt.axis('tight')

def plot_at_time(u,T):
    n=np.argmin(np.abs(t-T))

    print(n,t[n])


    u_ = u[:,n]
    N= len(u_)//4

    u1 = u_[0:N]
    u2 = u_[N:2*N]
    u3 = u_[2*N:3*N]
    u4 = u_[3*N:4*N]

    plt.plot(u1,label='u1')
    plt.plot(u2,label='u2')
    plt.plot(u3,label='u3')
    plt.plot(u4,label='u4')
    
    plt.title('T='+str(T))
    plt.legend()

### Explicit

In [ ]:
#Test for dense output formula implementation
#Not the correct formulf or dense output!!!!
ssp104.b_dense = lambda the: np.array([0.1][0.1*the]*10)

Stepsize Control Based on Hairer Solving ODE II



In [ ]:
#dp5
N =100
x = np.linspace(0,1,num = N)
dx=1/(N) # no -1 because of periodic boundary



u0 = initial_ADP(N)




#dp5
#dt = 0.0031 #dt_pos

  #dt_feasible for fallback = False #imeadetly crashes

  #dt_feasible for num_fallback = 1 #imeadetly crashes

#dt = 0.0041 #dt_feasible for num_fallback = 2

#dt = 0.006 # stil feasible but shows strong distortions near start num_fallback= 3
#dt = 0.0061 #dt_feasible for num_fallback = 3

#dt = 0.0031 # dt_stable

dt = 0.036


#t,u,b,status = RK_variable_b(ssp104,dt,f_ADP,u0,t_final=50,b_fixed=False,fallback = True,num_fallback= 3,
#                      LP_opts = {'verbose_LP':False,'reduce':True,'feastol':1e-10},return_status=True,verbose=False)

#t,u,b = RK_variable_b(dp5,dt,f_ADP,u0,t_final=50,b_fixed=False,fallback = True,num_fallback= 2,solver='scipy_ip',
#                      LP_opts = {'reduce':True})


solver = Solver(rkm = ck5,
               dt = 0.02,#0.0225,
               t_final = 50,
               b_fixed=False,
               tol_neg=1e-10,
               tol_change = 1.2,
               p = [4,3],#,2],
               theta = [1],#[1,0.5,0.25,0.0125],
               solver = cp.MOSEK, 
               LP_opts = {'reduce':False},
               fail_on_requect=False)
"""
solver = Solver(rkm = ssp104,
               dt = 0.0225,
               t_final = 10,
               b_fixed=False,
               tol_neg=1e-8,
               tol_change = 5,
               p = [4,3,2],
               theta = [1],
               solver = cp.ECOS,
               LP_opts = {})
"""

problem_ADR = Problem(f=f_ADP,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

control = StepsizeControl(dt_min = 0,dt_max = np.infty,a_tol = 0.1,r_tol=0.001,f = dt_logic_PI_POS
                          ,tol_reqect = 2*0.1)


status,t,u,b,KK = RK_integrate(solver=solver,problem=problem_ADR,stepsize_control=control,verbose=True,dumpK=True)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
#plot_image((u<0)*1.,t) #highlighted negeative
plot_image(u,t)

In [ ]:
T= 28#8.82
plot_at_time(u,T)

In [ ]:
plt.plot(t[1:],b.T[1:,:]);
#plt.xlim([10,15])
plt.ylim([2.5,-1])

In [ ]:
print('min:',np.min(u))
print(u[u<0])
print(np.sum(u<0))

In [ ]:
#slot = [500,520]
#slot = [1100,1200]
#slot = [2000,2200]
#slot = [3900,len(status['b'])]
slot = [0,len(status['b'])]


#At first plot the stepsizes according to the evalautions
dt = np.array(status['dt_calc'])
tol_met = np.array(status['sc'])
neg = np.array(status['b'])
n= np.arange(len(dt))
plt.plot(n,dt,'-k')
plt.plot(n[tol_met=='m'],dt[tol_met=='m'],'bx',label ='steps')
plt.plot(n[tol_met!='m'],dt[tol_met!='m'],'rx',label ='tol not met')
plt.plot(n[neg=='r'],dt[neg=='r'],'ro',label = 'infeasible')
plt.xlim(slot)
plt.grid()
plt.legend()

plt.ylim([0,0.1])

#plot error and change in commparison
plt.figure()
change = np.array(status['change'])
error = np.array(status['error'])
old_min = np.array(status['old_min'])
plt.plot(change,'x',label = 'Change')
plt.plot(error,'x',label = 'Error')
plt.plot(old_min,'x' ,label = 'Min')
plt.xlim(slot)
plt.ylim([-0,5])
plt.grid()
plt.legend()

Apparently this is a comlex problem to controll. Even for the stepsize control based only on the error estimation i was no able to set it up in a manner that does not lead to a big number of step reqects. 
And like expected the encorporation of the change does not make it more stable. But if the weight for the change is not chosento large it still works fine.

# Code to test Stepsize control and other properties

In [ ]:
#I am interested inthe correlation between these points
change = np.array(status['change'])
error = np.array(status['error'])

old_min = np.array(status['old_min'])

tol_met = np.array(status['sc'])
neg = np.array(status['b'])

plt.plot(error,change,'x',label = 'regular step')
plt.xlabel('Error')
plt.ylabel('Change')

#plot step reqects and infeasible
plt.plot(error[tol_met!='m'],change[tol_met!='m'],'rx',label = 'tol not met')
plt.plot(error[neg=='r'],change[neg=='r'],'ro',label = 'infeasible')
#at the moment there are no because we do not limit the change

plt.xlim([-0.1,2])
plt.ylim([-0.1,2])
plt.grid()


plt.figure()
plt.plot(old_min,change,'x')
plt.xlabel('Old Min')
plt.ylabel('Change')

plt.ylim([-0.1,2])
plt.xlim([-0.02,0.001])
plt.grid()

In [ ]:
old_min = np.array(status['old_min'])
new_min = np.array(status['new_min'])

old_min[old_min == None] = 0
new_min[new_min == None] = 0

In [ ]:
print(np.sum(old_min > new_min))
print(np.sum(old_min < new_min))
print(np.sum(old_min < new_min-1e-16))

In [ ]:
the = 0.1
b_dense =  lambda the: np.array([the-(1-0.1)*the**2]+[0.1*the**2]*9)

In [ ]:
(Q,rhs) = OrderCond(ssp104.A,ssp104.c,order = 2,theta=0.5)

In [ ]:
Q@b_dense(0.5) - rhs

In [ ]:
b_dense(0.5)

In [ ]:
dp5.plot_PI_controller_stability()

In [ ]:
ck5.plot_PI_controller_stability()

In [ ]:
chng = np.array(status['change'])
hist, bin_edges = np.histogram(chng[chng != None])
plt.bar(bin_edges[:-1], hist, width = 1,log = True)
plt.xlim(min(bin_edges), max(bin_edges))

In [ ]:
bin_edges

Error in Dcumentation plot_PI_controller_stability()
~/python/nodepy/nodepy/runge_kutta_method.py

$h_{n+1} = \left(\frac{\mathrm{TOL}}{\mathrm{err}_{n}}\right)^{\beta_1/k} \left(\frac{\mathrm{TOL}}{\mathrm{err}_{n-1}}\right)^{\beta_2/k} h_{n}$

In [ ]:
status['b']

In [ ]:
ck5.plot_I_controller_stability?